## Packages & helper functions

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from pymongo import MongoClient

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix

In [2]:
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 17}

matplotlib.rc('font', **font)

In [3]:
def plot_cm(y_true, y_pred, figsize=(6,5)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

In [4]:
def connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%d@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [5]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

## 1 - Prepare data

In [6]:
raw_data = read_mongo('baochi', 'tuoitre')
raw_data.head(10)

,content,category
0,Võ sĩ MMA bị knock-out lạ kỳ vì... 'đụng đầu' ...,Thể thao
1,"Thi đấu từ máy bay ở độ cao 6.096m, ngôi sao 6...",Thể thao
2,Ấn Độ yêu cầu mạng xã hội gỡ cụm từ 'biến thể ...,Công nghệ
3,Tay đua 19 tuổi Jason Dupasquier qua đời sau t...,Thể thao
4,Cảnh sát Ấn Độ 'cảnh báo' Công ty Twitter TTO ...,Công nghệ
5,Sủng Cỏ thiên đường hoang sơ nơi cửa vịnh Đà N...,Du lịch
6,"Giận liên đoàn, CĐV Indonesia không vào sân cổ...",Thể thao
7,Google bỏ cookie theo dõi người dùng là để 'là...,Công nghệ
8,"Võ sĩ hạng nặng đánh nhau trên sàn đấu, khán g...",Thể thao
9,Bánh trứng thời nghèo khó Dan Bing bỗng len lỏ...,Du lịch


In [7]:
raw_data.groupby('category').size()

category
Công nghệ    1579
Du lịch      2535
Thể thao     3058
dtype: int64

In [8]:
data = raw_data[raw_data.category == 'Thể thao']
# data = data.head(50)

## 2 - Apply TF-IDF on text

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['content']).toarray()
vocab = vectorizer.get_feature_names()

In [10]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13518198, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## 3 - Cluster data by k-means

### 3.1. Tìm số clusters tối ưu bằng   Elbow Method

In [11]:
wcss = []
num = 200
for i in range(1,num): 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 500, n_init = 10, random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,num), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

KeyboardInterrupt: 